now let's try 100 epochs but this time save the training loss. now let's try 10 fold

In [1]:
%%capture output
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation, BatchNormalization
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix
import pickle
import matplotlib.pyplot as plt
import os
from keras.applications import ResNet50
from tqdm import tqdm  # For progress bar

# Paths to the base directory containing all folds
base_data_dir = '/home/wangg/REU-Hearing-Loss-Project-1/machine learning/allFolds/10folds - 80-20 train test split (includes all subjects)'

# Image dimensions
img_height, img_width = 224, 224

# Define batch size
batch_size = 32
for fold_number in range(1, 11):
    # Paths to training and testing data for the current fold
    training_data_path = os.path.join(base_data_dir, f'fold{fold_number}/Training')
    testing_data_path = os.path.join(base_data_dir, f'fold{fold_number}/Testing')

    # Create data generators for training and testing with data augmentation
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        horizontal_flip=True,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2
    )
    # Create data generators for training and testing
    # train_datagen = ImageDataGenerator(rescale=1./255)
    test_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
        training_data_path,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical'  # bc we have two classes
    )

# # Print the class names
# class_names = list(train_generator.class_indices.keys())
# print("Class Names:", class_names)

    test_generator = test_datagen.flow_from_directory(
        testing_data_path,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical'
    )

    # Load the pre-trained ResNet-50 model with weights from ImageNet
    base_model = ResNet50(
        include_top=False, # don't want to include the top layer because we will define our own input/output layer
        weights='imagenet',
        input_tensor=None,
        input_shape=(img_height, img_width, 3),
        # pooling=None,
        pooling= 'max',
        classes=2,
    )
    # Freeze only the convolutional layers of the pre-trained model
    for layer in base_model.layers:
        if isinstance(layer, keras.layers.Conv2D):
            layer.trainable = False

            
    # Add custom classification layers on top of ResNet-50
    # these new layers and specifically the 512 neurons are the ones that will be learning the new weights 
    x = Flatten()(base_model.output)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(2, activation='softmax')(x)  # Assuming 2 classes: Healthy and Hearing Impaired

    # Create the final model
    model = keras.Model(base_model.input, x)

    # Compile the model
    model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

    # Train the model
    history = model.fit(train_generator, epochs=100)

    # Train the model and save training history
    history = model.fit(train_generator, epochs=100, verbose=0)

    # Save training loss values to a file
    losses_filename = f'pretrained_losses_fold_{fold_number}.txt'
    with open(losses_filename, 'w') as losses_file:
        losses_file.write("Epoch\tTraining Loss\tTraining Accuracy\n")
        for epoch in range(len(history.history['loss'])):
            losses_file.write(f"{epoch + 1}\t{history.history['loss'][epoch]}\t{history.history['accuracy'][epoch]}\n")

    # Evaluate the model on the testing data
    test_loss, test_acc = model.evaluate(test_generator)
    print(f"\nFold {fold_number} - Test Accuracy: {test_acc}")

    # Get true labels and predicted labels
    predictions = model.predict(test_generator)
    predicted_labels = np.argmax(predictions, axis=1)
    true_labels = test_generator.classes

    # Calculate precision, recall, and f1-score
    precision, recall, f1_score, _ = precision_recall_fscore_support(true_labels, predicted_labels, average='weighted')

    # Save classification results to a file
    results_filename = f'pretrained_classification_results_fold_{fold_number}.txt'
    with open(results_filename, 'w') as results_file:
        results_file.write("Image Name\tTrue Label\tPredicted Label\n")
        for i in range(len(test_generator.filenames)):
            image_name = os.path.basename(test_generator.filenames[i])
            true_label = true_labels[i]
            predicted_label = predicted_labels[i]
            results_file.write(f"{image_name}\t{true_label}\t{predicted_label}\n")

    # Save metrics to a text file
    metrics_filename = f'pretrained_classification_metrics_fold_{fold_number}.txt'
    with open(metrics_filename, 'w') as file:
        file.write(f'Test Accuracy: {test_acc}\n')
        file.write(f'Precision: {precision}\n')
        file.write(f'Recall: {recall}\n')
        file.write(f'F1 Score: {f1_score}\n')

2024-01-26 16:25:56.266843: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-26 16:26:01.686460: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2024-01-26 16:26:08.018173: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
